# pose estimation with MoveNet

## 0. Installing and Importing Dependencies

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

2023-06-27 09:51:05.381560: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1. Loading model

[Link to model and official docs](https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/3)

website -> model format -> TFLite (movenet/singlepose/lightning) -> model card

*  MoveNet.SinglePose.Lightning: A lower capacity model that can run >50FPS on most modern laptops while achieving good performance.
* MoveNet.SinglePose.Thunder: A higher capacity model that performs better prediction
quality while still achieving real-time (>30FPS) speed. Naturally, thunder will lag behind
the lightning, but it will pack more of a punch.

gonna stick with Lightening first; download it and have it in the current directory

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


## Detection

### Taking live-streaming camera input

In [3]:
# taking input from webcam
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('MoveNet Lightning', frame)

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@17.166] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


### Taking image input
